In [1]:
# Using SSH via Python

import paramiko
from getpass import getpass
import os
import sys

# Create the file prompt
theFile = input("Please enter the desired file for the output ")

# Little test to make sure there is actual input
if theFile == "":
    sys.exit("No response detected. Stopping code.")
else:
    pass

# Check that theFile exists in local directory
if os.path.exists(theFile) == True:
    changeFile = input('The specified file already exists.  |  Would you like to replace this file removing the previous one?(yes/no) ')

    # Following if,elif,else statement determines what to do with theFile based on users input
    if changeFile.lower() == "no":
        sys.exit("Okay the file will remain intact and the code will stop.")

    elif changeFile.lower() == "yes":
        os.remove(theFile)
        pass

    else:
        sys.exit("No response detected. Stopping code.")
        
# If it does not exist continue like usual
else:
    pass

# Create the password prompt
thePass = getpass(prompt="Please enter your SSH password: ")

# Host information
host = "192.168.6.71"
port = 2222
username = "christopher.mchugh"
password = thePass

try:

    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh.connect(host, port, username, password)

except paramiko.AuthenticationException:
    print("Authentication Failed.")


In [3]:
# Start SFTP session
sftp = ssh.open_sftp()

# Upload the script to the remote device
sftp.put('C:/Users/Christopher/Desktop/School/SYS_320_SP22/Week07/homework/yara.txt',"/home/christopher.mchugh/yara.txt")

<SFTPAttributes: [ size=0 uid=1286604063 gid=1286603265 mode=0o100644 atime=1646515049 mtime=1646515049 ]>

In [2]:
sftp = ssh.open_sftp()
# Run a command
command = 'sudo -S kraken --rules yara.txt --folder /usr/bin --folder  --folder /usr/sbin/   --folder /usr/local/bin  --folder /sbin  --folder /usr/local/sbin  --folder /bin >> '+theFile

# Get the output from the command
stdin, stdout, stderr = ssh.exec_command(command)

# input password to run command as sudo, including a newline
# character that provides "Enter" as if you were typing it on a keyboard
stdin.write(thePass+"\n")

# Get results from stdout
lines = stdout.readlines()
#print(lines)

# Convert the list to a string
output = ''.join(lines)

# Save the output to a file
with open(theFile, 'a') as f:
    f.write(output)
#print(cmd_output)
sftp.get('/home/christopher.mchugh/'+theFile,theFile)
ssh.close()


In [12]:
# All exploitation commands
cmds = {'blind_files' : ['cat /etc/resolv.conf','cat /etc/issue','cat etc/passwd','sudo -s cat /etc/shadow','cat /home/'+username+'.bash_history'],
        'system' : ['uname -a','ps aux','id','arch','w','who -a','perl -v','df -k','mount','last -a','lastlog','dmesg','lspci','lscpu','lshw','cat /proc/cpuinfo','cat/proc/meminfo'],
        'networking' : ['hostname -f','ip addr show','ip ro show','sudo -s iptables -L -n -v','sudo -s iptables-save','cat /proc/net/*'],
        'user_accounts' : ['cat /etc/passwd','sudo -s cat /etc/shadow','cat /etc/group','getemt passwd','getent group'],
        'obt_user_info' : ['ls -alh /home/*/','ls -alh /home/*/.ssh/','sudo -s cat /home/*/.viminfo','sudo -l','crontab -l'],
        'credentials' : [''],
        'configs' : ["sudo -s ls -aRl /etc/ * awk '$1 ~ /w.$/' * grep -v lwrx 2> sudo /dev/nullte",'cat /etc/group','cat /etc/hosts','cat /etc/crontab','cat /etc/sysctl.conf','cat /etc/resolv.conf','cat /etc/ldap/ldap.conf','cat /etc/fstab'],
        'determine_distro' : ['uname -a','lsb_release -d','cat /etc/os-release','cat /etc/issue','cat /etc/*release','cat /etc/debian_version','arch'],
        'installed_packages' : ['dpkg -l','dpkg -l | grep -i "linux-image"','dpkg --get-selections'],
        'package_sources' : ['cat /etc/apt/sources.list'],
        'find_important_files' : ['ls -alR | grep ^d','find /var -type -d','ls -dl `find /var -type d`','find /var -type d` | grep -v root','find /var ! -user root -type d -ls','sudo -s find / -perm -4000','ls -alhtr /mnt','ls -alhtr /media','ls -alhtr /tmp','ls -alhtr /home',"sudo -s find /home -type f -iname '.*history'","find /sbin /user/sbin /opt /lib `echo $PATH | 'sed s/:/ /g'` -perm /6000 -ls"],
}
